In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

In [2]:
train  = pd.read_csv('competition_data/train.csv')
test  = pd.read_csv('competition_data/test.csv')

In [3]:
train

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,...,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,...,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,...,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1
14998,14998,5.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,5.0,...,1.0,29,1.0,12.0,4.0,2.0,2.0,2.0,1.0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         15000 non-null  int64  
 1   Q1            14959 non-null  float64
 2   Q2            14931 non-null  float64
 3   Q3            14950 non-null  float64
 4   Q4            14929 non-null  float64
 5   Q5            14962 non-null  float64
 6   Q6            14952 non-null  float64
 7   Q7            14924 non-null  float64
 8   Q8            14952 non-null  float64
 9   Q9            14944 non-null  float64
 10  Q10           14928 non-null  float64
 11  Q11           14941 non-null  float64
 12  Q12           14933 non-null  float64
 13  Q13           14960 non-null  float64
 14  Q14           14964 non-null  float64
 15  Q15           14955 non-null  float64
 16  Q16           14967 non-null  float64
 17  Q17           14963 non-null  float64
 18  Q18           14937 non-nu

In [5]:
train = train.drop(['index', 'country'],axis = 1)
test =test.drop(['index', 'country'],axis = 1)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 68 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            14959 non-null  float64
 1   Q2            14931 non-null  float64
 2   Q3            14950 non-null  float64
 3   Q4            14929 non-null  float64
 4   Q5            14962 non-null  float64
 5   Q6            14952 non-null  float64
 6   Q7            14924 non-null  float64
 7   Q8            14952 non-null  float64
 8   Q9            14944 non-null  float64
 9   Q10           14928 non-null  float64
 10  Q11           14941 non-null  float64
 11  Q12           14933 non-null  float64
 12  Q13           14960 non-null  float64
 13  Q14           14964 non-null  float64
 14  Q15           14955 non-null  float64
 15  Q16           14967 non-null  float64
 16  Q17           14963 non-null  float64
 17  Q18           14937 non-null  float64
 18  Q19           14947 non-nu

In [7]:
#train을 target과 feature로 나눠줍니다.
train_ = train.dropna(axis=0)


In [8]:
train_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12796 entries, 0 to 14999
Data columns (total 68 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            12796 non-null  float64
 1   Q2            12796 non-null  float64
 2   Q3            12796 non-null  float64
 3   Q4            12796 non-null  float64
 4   Q5            12796 non-null  float64
 5   Q6            12796 non-null  float64
 6   Q7            12796 non-null  float64
 7   Q8            12796 non-null  float64
 8   Q9            12796 non-null  float64
 9   Q10           12796 non-null  float64
 10  Q11           12796 non-null  float64
 11  Q12           12796 non-null  float64
 12  Q13           12796 non-null  float64
 13  Q14           12796 non-null  float64
 14  Q15           12796 non-null  float64
 15  Q16           12796 non-null  float64
 16  Q17           12796 non-null  float64
 17  Q18           12796 non-null  float64
 18  Q19           12796 non-nu

In [9]:
train_S = StandardScaler().fit_transform(train_.iloc[:, :-1])
train_S = pd.DataFrame(train_S)
train_S

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,66
0,-2.729377,0.796258,0.759147,0.987636,-2.405805,0.267537,0.766632,0.860341,-2.607258,-1.086667,...,2.334647,-0.718286,-0.020718,1.712065,2.340299,1.777416,0.766546,-0.383813,0.047441,0.253562
1,0.025041,-0.062841,-0.229459,0.161259,0.113468,1.124460,-0.181818,0.088976,-0.817954,-1.086667,...,0.535325,-0.718286,0.063774,-0.383953,-0.597213,-0.691696,-1.304553,1.794136,0.047441,0.253562
2,0.025041,0.796258,0.759147,0.161259,-0.726290,1.124460,0.766632,0.860341,0.076698,-0.122983,...,-1.263997,1.392204,0.046293,-0.383953,-0.597213,0.131341,0.766546,3.972084,0.047441,0.253562
3,0.025041,-0.062841,-0.229459,-1.491497,0.113468,-0.589385,-1.130269,0.860341,-0.817954,-0.122983,...,-1.263997,-0.718286,-0.029459,1.712065,-0.890964,-0.691696,0.766546,-0.383813,-0.030024,0.253562
4,0.025041,-0.062841,-0.229459,0.161259,-0.726290,-0.589385,-0.181818,-1.453754,-0.817954,-0.122983,...,0.535325,1.392204,-0.026545,1.712065,2.340299,-0.691696,0.766546,-0.383813,-0.068756,0.253562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12791,-1.811237,0.796258,-0.229459,-0.665119,-0.726290,0.267537,-0.181818,0.088976,-0.817954,-0.122983,...,0.535325,-0.718286,-0.029459,-0.383953,-0.890964,0.954379,0.766546,-0.383813,0.008708,0.253562
12792,0.943181,-0.062841,0.759147,0.161259,0.113468,1.124460,0.766632,0.088976,0.076698,0.840700,...,0.535325,1.392204,0.052120,-0.383953,-0.303462,-0.691696,-1.304553,1.794136,0.008708,0.253562
12793,0.025041,0.796258,0.759147,0.987636,0.953226,1.124460,0.766632,0.860341,0.076698,0.840700,...,0.535325,-0.718286,-0.020718,-0.383953,-0.890964,0.131341,-1.304553,-0.383813,0.008708,-3.943816
12794,0.943181,0.796258,-0.229459,0.987636,0.953226,1.124460,0.766632,-2.225119,0.971351,0.840700,...,0.535325,-0.718286,0.005503,-0.383953,2.340299,1.777416,0.766546,1.794136,-0.030024,-3.943816


In [10]:
train_S.columns=['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8'
               ,'Q9','Q10','Q11','Q12','Q13','Q14','Q15','Q16'
               ,'Q17','Q18','Q19','Q20','Q21','Q22','Q23','Q24'
               ,'Q25','Q26','introelapse','testelapse','surveyelapse'
                ,'TIPI1','TIPI2','TIPI3'
               ,'TIPI4','TIPI5','TIPI6','TIPI7','TIPI8','TIPI9','TIPI10'
                ,'VCL1','VCL2','VCL3','VCL4','VCL5','VCL6','VCL7'
               ,'VCL8','VCL9','VCL10','VCL11','VCL12','VCL13','VCL14'
               ,'VCL15','VCL16','education','urban','gender','engnat','age'
               ,'hand','religion','orientation','voted','married','familysize'
                ,'ASD']

In [11]:
train_S

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD
0,-2.729377,0.796258,0.759147,0.987636,-2.405805,0.267537,0.766632,0.860341,-2.607258,-1.086667,...,2.334647,-0.718286,-0.020718,1.712065,2.340299,1.777416,0.766546,-0.383813,0.047441,0.253562
1,0.025041,-0.062841,-0.229459,0.161259,0.113468,1.124460,-0.181818,0.088976,-0.817954,-1.086667,...,0.535325,-0.718286,0.063774,-0.383953,-0.597213,-0.691696,-1.304553,1.794136,0.047441,0.253562
2,0.025041,0.796258,0.759147,0.161259,-0.726290,1.124460,0.766632,0.860341,0.076698,-0.122983,...,-1.263997,1.392204,0.046293,-0.383953,-0.597213,0.131341,0.766546,3.972084,0.047441,0.253562
3,0.025041,-0.062841,-0.229459,-1.491497,0.113468,-0.589385,-1.130269,0.860341,-0.817954,-0.122983,...,-1.263997,-0.718286,-0.029459,1.712065,-0.890964,-0.691696,0.766546,-0.383813,-0.030024,0.253562
4,0.025041,-0.062841,-0.229459,0.161259,-0.726290,-0.589385,-0.181818,-1.453754,-0.817954,-0.122983,...,0.535325,1.392204,-0.026545,1.712065,2.340299,-0.691696,0.766546,-0.383813,-0.068756,0.253562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12791,-1.811237,0.796258,-0.229459,-0.665119,-0.726290,0.267537,-0.181818,0.088976,-0.817954,-0.122983,...,0.535325,-0.718286,-0.029459,-0.383953,-0.890964,0.954379,0.766546,-0.383813,0.008708,0.253562
12792,0.943181,-0.062841,0.759147,0.161259,0.113468,1.124460,0.766632,0.088976,0.076698,0.840700,...,0.535325,1.392204,0.052120,-0.383953,-0.303462,-0.691696,-1.304553,1.794136,0.008708,0.253562
12793,0.025041,0.796258,0.759147,0.987636,0.953226,1.124460,0.766632,0.860341,0.076698,0.840700,...,0.535325,-0.718286,-0.020718,-0.383953,-0.890964,0.131341,-1.304553,-0.383813,0.008708,-3.943816
12794,0.943181,0.796258,-0.229459,0.987636,0.953226,1.124460,0.766632,-2.225119,0.971351,0.840700,...,0.535325,-0.718286,0.005503,-0.383953,2.340299,1.777416,0.766546,1.794136,-0.030024,-3.943816


In [12]:
train_S['nerdiness']=train_['nerdiness']

In [13]:
train_.iloc[:, -1]=train_['nerdiness']

In [14]:
train_x = train_.drop(['nerdiness'],axis=1)
train_y = train_['nerdiness']

In [ ]:
start_time=time.time()
lgb= lgb.LGBMClassifier(
            
        )
parametes= {
    'num_leaves':[20,40,60,80,100],'min_child_samples':[5,10,15],
    'max_depth':[-1,5,10,20],'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]
}
clf=RandomizedSearchCV(lgb,parametes,scoring='accuracy',n_iter=100)
clf.fit(X=train_x,y=train_y)
print(clf.best_params_)
end_time=time.time()
print('소요 시간 : ')
print(end-start)

In [ ]:
#lgbm_clf.fit(train_x, train_y)
lgbm_pred = clf.predict(test)

In [ ]:
submission = pd.read_csv('competition_data/sample_submission.csv')

submission

In [ ]:
submission["nerdiness"] = lgbm_pred

In [ ]:
submission

In [ ]:
submission.to_csv("baseline_notNA-withSC-Grid.csv", index = False)